In [ ]:

import pandas as pd
import numpy as np
from sklearn.utils import resample
from rapidfuzz import process,fuzz
import unidecode

In [ ]:
data=pd.read_csv('Data.csv')
#cleaning process

#1) chech for missing values and calculating the percentage of missing values per column
print(data.isnull().sum())
print((data.isnull().sum()/len(data))*100)

#data is not having any missing values


In [ ]:
#2) check for duplicated values
data.duplicated().any() # returns True if there are any duplicated rows, False otherwise
data.duplicated().sum() # returns the number of duplicated rows
data[data.duplicated()] # returns the duplicated rows

#data is not having any duplicated values

In [ ]:
#3) Handle inconsistent data entries
#check if the columns is in the right data type

print(data.dtypes)
# we have Date as object we need to convert it to datetime64

In [ ]:
data["Date"]=pd.to_datetime(data['Date'],errors='coerce')
data.to_csv('Data.csv',index=False)
print(data.dtypes)
# all columns do not have un unified values 

In [ ]:
#Normalize categorical values

data['Date'].value_counts()     

In [ ]:
data['Product Category'].value_counts()

In [ ]:
choices = data['Product Name'].unique()
similar_pairs=[]
for product in choices:
    matches = process.extract(product, choices, limit=None, scorer=fuzz.partial_ratio)
    for match, score,index in matches:
        if score >=75 and product !=match:
            pair=tuple(sorted([product,match]))
            if pair not in similar_pairs:
                similar_pairs.append((pair[0],pair[1],score)) 

for p1,p2,score in similar_pairs:
    print(f"'{p1}' and '{p2}' have a similarity score of {score}") 

data['Product Name'].replace('MacBook Pro 16-inch','Apple MacBook Pro 16-inch',inplace=True)
data['Product Name'].replace('Nike Air Force 1','Nike Air Force 1 Sneakers',inplace=True)
data['Product Name'].replace('Adidas Ultraboost Shoes','Adidas Ultraboost Running Shoes',inplace=True)
data['Product Name'].replace('Yeti Rambler Tumbler','Yeti Rambler 20 oz Tumbler',inplace=True)
data.to_csv('Data.csv', index=False)

In [ ]:
data['Region'].value_counts()

In [ ]:
data['Payment Method'].value_counts()

In [ ]:
data['Product Name']=data['Product Name'].str.strip() #this removes the space at the beginning and end of the string
data.to_csv('Data.csv', index=False)

In [ ]:
#check for non-ASCII characters in 'Product Name' column
import unicodedata


mask=data['Product Name'].apply(lambda x: not all(ord(c) <128 for c in str(x))) 
#this returns a boolean series where True indicates the presence of non-ASCII characters
data[mask]

#function to normalize text by removing accents and special characters
def normalize_text(text): 
    return unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8') 
data['Product Name'] = data['Product Name'].apply(normalize_text)
data.to_csv('Data.csv', index=False)


In [ ]:
#check for invalid values like -ve values or unreasonable values in the numerical columns
invalid_unitSolds=data[data['Units Sold']<0]
invalid_unitPrice=data[data['Unit Price']<0]
invalid_totalRevenue=data[data['Total Revenue']<0]

print(invalid_unitSolds)
print(invalid_unitPrice)
print(invalid_totalRevenue)    

In [94]:
#4) Handling outliers

numeric_cols = data[['Units Sold', 'Unit Price', 'Total Revenue']] 

for col in numeric_cols:
    Q1=data[col].quantile(0.25)
    Q3=data[col].quantile(0.75)
    IQR=Q3-Q1
    lower=Q1 - 1.5 * IQR
    upper=Q3 + 1.5 * IQR
    outliers=data[(data[col]<lower) | (data[col]>upper)]
    #here we gave the data[col] col now is year and is now a series
    #the series is undergoing a condition it will check if it's true and retrun a series of boolean
    #now outlier is a dataframe that contains all the rows that have outliers in that specific column
    if not outliers.empty:
        print(f"\nColumn '{col}' has outliers:")
        print(outliers[[col]])

#there is no outliers in the dataset


Column 'Units Sold' has outliers:
    Units Sold
62          10

Column 'Unit Price' has outliers:
     Unit Price
0        999.99
6       2499.99
7        599.99
12       749.99
35      1895.00
37       799.99
66      1199.99
72      1599.99
73       899.99
85      1299.99
95       999.99
102     3899.99
120      599.99
132     1299.99
138     1499.99
156      899.99
162     1199.99
168      699.99
173      699.99
180      649.99
210      749.99
216     2399.00
221      599.99
229      599.00

Column 'Total Revenue' has outliers:
     Total Revenue
0          1999.98
6          2499.99
7          1199.98
12         1499.98
18          999.98
35         1895.00
36         1199.97
37         1599.98
42         1199.97
66         1199.99
72         1599.99
85         2599.98
95          999.99
102        3899.99
120        1199.98
132        1299.99
138        1499.99
162        1199.99
216        2399.00
